In [1]:
import optuna 
import xgboost as xgb
from catboost import CatBoostRegressor
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import math
import sklearn
from sklearn.metrics import mean_squared_error
import operator
import catboost as ctb

In [2]:
train = pd.read_csv('..\\kaggle_data\\train.csv')
test = pd.read_csv('..\\kaggle_data\\test.csv')

In [3]:
cat_features = [
    "cat0", "cat1", "cat2", "cat3", "cat4", "cat5", "cat6", "cat7", 
    "cat8", "cat9"
]

cont_features = [
    "cont0", "cont1", "cont2", "cont3", "cont4",
    "cont5", "cont6", "cont7", "cont8", "cont9", "cont10", 
    "cont11", "cont12", "cont13"
]
alpha = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z' ]
alpha_conversion = {}
for index in range(0, len(alpha)):
    alpha_conversion[alpha[index]] = index / 29
alpha_conversion

for cat in cat_features:
    train[cat].replace(alpha_conversion, inplace=True)
    train[cat] = train[cat].astype('float')
train = train.drop(['id'], axis=1)
train.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,0.000000,0.034483,0.0,0.000000,0.034483,0.103448,0.0,0.137931,0.068966,0.275862,...,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
1,0.034483,0.000000,0.0,0.000000,0.034483,0.034483,0.0,0.137931,0.000000,0.172414,...,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
2,0.000000,0.000000,0.0,0.068966,0.034483,0.103448,0.0,0.034483,0.068966,0.448276,...,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
3,0.000000,0.000000,0.0,0.068966,0.034483,0.103448,0.0,0.137931,0.206897,0.344828,...,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
4,0.000000,0.034483,0.0,0.000000,0.034483,0.034483,0.0,0.137931,0.068966,0.172414,...,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974


In [4]:
columns = [col for col in train.columns.to_list() if col not in ['id','target']]
data=train[columns]
target=train['target']

In [7]:
def objective(trial,data=data,target=target):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    param = {
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': trial.suggest_categorical('n_estimators', [4000, 5000]),
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17,20]),
        'random_state': trial.suggest_categorical('random_state', [24, 48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    model = xgb.XGBRegressor(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

In [8]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=500)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-02-26 15:09:44,759] A new study created in memory with name: no-name-f6618d70-a6dc-4bb9-9126-158493062afd
[I 2021-02-26 15:10:17,145] Trial 0 finished with value: 0.8444410212147839 and parameters: {'lambda': 6.155984984542258, 'alpha': 0.17747533932242732, 'colsample_bytree': 0.9, 'subsample': 0.4, 'learning_rate': 0.01, 'n_estimators': 5000, 'max_depth': 7, 'random_state': 48, 'min_child_weight': 5}. Best is trial 0 with value: 0.8444410212147839.
[I 2021-02-26 15:11:17,301] Trial 1 finished with value: 0.8436517945091837 and parameters: {'lambda': 0.7573186802003511, 'alpha': 0.1605562637729797, 'colsample_bytree': 0.3, 'subsample': 0.5, 'learning_rate': 0.012, 'n_estimators': 4000, 'max_depth': 15, 'random_state': 48, 'min_child_weight': 144}. Best is trial 1 with value: 0.8436517945091837.
[I 2021-02-26 15:14:21,689] Trial 2 finished with value: 0.8496933682980573 and parameters: {'lambda': 0.46070163869483427, 'alpha': 0.0014126423270059414, 'colsample_bytree': 0.9, 'subs

KeyboardInterrupt: 

In [98]:
Best_trial= {'lambda': 0.04324261140518615, 'alpha': 0.002255618014464506, 'colsample_bytree': 0.3, 'subsample': 0.8, 'learning_rate': 0.008, 'n_estimators': 4000, 'max_depth': 9, 'random_state': 2020, 'min_child_weight': 206,
             'tree_method':'gpu_hist'}

In [ ]:
best_trial = study.best_trial.params
#best_trial['tree_method'] = 'gpu_hist'

In [99]:
preds = np.zeros(test.shape[0])
kf = KFold(n_splits=5,random_state=48,shuffle=True)
rmse=[]  # list contains rmse for each fold
n=0
for trn_idx, test_idx in kf.split(train[columns],train['target']):
    print(trn_idx) 
    print(test_idx)
    break
    X_tr,X_val=train[columns].iloc[trn_idx],train[columns].iloc[test_idx]
    y_tr,y_val=train['target'].iloc[trn_idx],train['target'].iloc[test_idx]
    model = xgb.XGBRegressor(**Best_trial)
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
    preds+=model.predict(test[columns])/kf.n_splits
    rmse.append(mean_squared_error(y_val, model.predict(X_val), squared=False))
    print(n+1,rmse[n])
    n+=1

[     1      2      3 ... 299997 299998 299999]
[     0      4      6 ... 299991 299993 299994]


In [102]:
def create_folds(df, n_s=5, n_grp=None):
    df['Fold'] = -1
    
    if n_grp is None:
        skf = KFold(n_splits=n_s)
        target = df.target
    else:
        skf = StratifiedKFold(n_splits=n_s)
        df['grp'] = pd.cut(df.target, n_grp, labels=False)
        target = df.grp
    
    for fold_no, (t, v) in enumerate(skf.split(target, target)):
        df.loc[v, 'Fold'] = fold_no
    return df

In [104]:
train = create_folds(train, n_s=10, n_grp=1000)
preds = np.zeros(test.shape[0])
rmse=[]  # list contains rmse for each fold
n=0

for fold in np.sort(train.Fold.unique())[1:]:
    working_set = train[train.Fold==fold]
    
    x = working_set.drop(["target", "grp"],axis=1)
    y = working_set.target

    model = ctb.CatBoostRegressor(iterations=5000)
    model.fit(x,y,verbose=False)

    preds+=model.predict(test[columns])/kf.n_splits
    rmse.append(mean_squared_error(y_val, model.predict(X_val), squared=False))
    print(n+1,rmse[n])
    n+=1


c:\users\ryans\source\repos\venvs\python375\lib\site-packages\sklearn\model_selection\_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=0]="A": Cannot convert 'b'A'' to float

In [ ]:
from datetime import datetime
dt_string = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")

sub['target']=preds

print(sub.head())

sub.to_csv(f'..\\kaggle_data\\{dt_string}_submission.csv', index=False)